# Task 1: Improving the accuracy of our logistic regression on MNIST

### Done.
*Accuracy > 0.97 (0.9714)*

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time

# Define paramaters for the model
learning_rate = 0.001
batch_size = 256
n_epochs = 100
reg = 0.001

In [2]:
# Step 1: Read in data
mnist = input_data.read_data_sets('/data/mnist', one_hot=True)

# Step 2: create placeholders for features and labels
X = tf.placeholder(tf.float32, [batch_size, 784], name='image')
Y = tf.placeholder(tf.float32, [batch_size, 10], name='label')

# Step 3: create weights and bias
w1 = tf.Variable(tf.random_normal([784, 100], stddev=0.1) * tf.sqrt(2.) / batch_size , name='weight1')
b1 = tf.Variable(tf.zeros([1, 100]), name='bias1')

w2 = tf.Variable(tf.random_normal([100, 10], stddev=0.1) * tf.sqrt(2.) / batch_size , name='weight2')
b2 = tf.Variable(tf.zeros([1, 10]), name='bias2')

# Step 4: build model
# DO NOT DO SOFTMAX HERE
h1 = tf.nn.relu(tf.matmul(X, w1) + b1)
logits = tf.matmul(h1, w2) + b2

# Step 5: define loss function
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
loss = tf.reduce_mean(entropy) + reg * (tf.reduce_sum(w1 ** 2) + tf.reduce_sum(w2 ** 2))

# Step 6: define training op
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

Extracting /data/mnist\train-images-idx3-ubyte.gz
Extracting /data/mnist\train-labels-idx1-ubyte.gz
Extracting /data/mnist\t10k-images-idx3-ubyte.gz
Extracting /data/mnist\t10k-labels-idx1-ubyte.gz


In [3]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs): # train the model n_epochs times
        total_loss = 0

        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            # TO-DO: run optimizer + fetch loss_batch
            __, loss_batch = sess.run([optimizer, loss], feed_dict={X:X_batch, Y:Y_batch})
            total_loss += loss_batch
        if (i + 1) % 10 == 0:
            print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

    print('Total time: {0} seconds'.format(time.time() - start_time))
    print('Optimization Finished!')

    # test the model
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, logits], feed_dict={X: X_batch, Y:Y_batch})
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
        total_correct_preds += sess.run(accuracy)

    print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

Average loss epoch 9: 0.2603639990926903
Average loss epoch 19: 0.23800325546866266
Average loss epoch 29: 0.2320742225535562
Average loss epoch 39: 0.22983228540587647
Average loss epoch 49: 0.22798051768652747
Average loss epoch 59: 0.22700998444702022
Average loss epoch 69: 0.22633846285187195
Average loss epoch 79: 0.22503264352818517
Average loss epoch 89: 0.22473169884113509
Average loss epoch 99: 0.2246203877937014
Total time: 63.40707588195801 seconds
Optimization Finished!
Accuracy 0.9714
